In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [6]:
data_set=pd.read_csv('train.csv')

In [7]:
test_set=pd.read_csv('test.csv')

In [8]:
labelencoder_X=LabelEncoder()

In [22]:
def data_prep(train,test):
    all_data=pd.concat([train.drop('Survived',1),test])
    
    #Feature Title, Master, miss and other titles give us the indication that the person was young, other titles looks that the person was relativly old
    all_data['Title']=all_data['Name'].map(lambda complete_name: complete_name.split(',')[1].split('.')[0].strip())
    titles=set(all_data['Title'])
    temp_data=all_data[:len(train)]
    titles_mapping=dict(zip(titles,range(0,len(titles)+1)))
    temp_data['Numeric_title']=temp_data.apply(lambda passanger:titles_mapping[passanger.Title],axis=1).astype(int)
    all_data['Numeric_title']=all_data.apply(lambda passanger:titles_mapping[passanger.Title],axis=1).astype(int)
    #Fill unkonw embarcation with the mode of the embarked zone of the train data
    all_data.loc[all_data.Embarked.isnull(),'Embarked']=train.Embarked.mode()[0]
    #check why next line doesnt return int, instead return the letter
    #all_data['Embarket']=all_data['Embarked'].map({'C': 0, 'S': 1, 'Q': 2}).astype(int)
    all_data['Embarked']=labelencoder_X.fit_transform(all_data['Embarked'])
    embarket_place=sorted(all_data['Embarked'].unique(), key=lambda x: x)
    #Now let's fill missing age with the average of each title class but only using the train
    average_age_title={}
    for i in range(0,len(titles_mapping)):
        average_age_title[i]=np.median(train.loc[temp_data.Numeric_title==i,('Age')].dropna())
    all_data.loc[all_data.Age.isnull(),('Age')]=all_data[all_data.Age.isnull()].apply(lambda element: average_age_title.get(element.Numeric_title),axis=1)
    #From histogram people up to 10 have better prob to survive, for people with more than 63 is the opposite
    all_data['Young']=all_data.apply(lambda element: int(element.Age<=10),axis=1)
    all_data['Late_adulthood']=all_data.apply(lambda element: int(element.Age>=63),axis=1)
    #family size, non_including his/her-self
    all_data['Family_size']=all_data['SibSp']+all_data['Parch']
    #If the person is not-alone has better chances to survive
    all_data['Alone']=all_data.apply(lambda element: int((element.SibSp+element.Parch)==0),axis=1)
    #young+alone seem like has less probablity to live, but maybe is not so important since in the train only 1 is young and alone
    all_data['Young_alone']=all_data.apply(lambda element: (element.Young==1)and(element.Alone==1),axis=1).astype(int)
    #Also bigger families lower prob to survive :(
    all_data['Big_family']=all_data.apply(lambda element: (element.SibSp>2)or(element.Parch>3),axis=1).astype(int)
    #Obviuslly sex has a big impact, female has better prob to survive, there are sklearn ways to do this but pandas is really nice
    all_data['Sex']=all_data['Sex'].map({'male': 0, 'female': 1}).astype(int)
    #There is some correlation between fare and embarket place so let's use the mean/median (need to pay attention to the skwekness) for each embarket place
    # however let;s do this in future versions
    
    average_fare_class=np.zeros([3,3])
    #there are 3 classes so let's use it
    for i in range(1,4):
        for j in range(1,4):
            average_fare_class[i-1][j-1]=(np.nanmedian((train.Pclass==i)&(train.Embarked==j)))
    all_data.loc[all_data.Fare.isnull(),('Fare')]=all_data[all_data.Fare.isnull()].apply(lambda element: average_fare_class[element.Pclass-1][element.Embarked-1],axis=1)
    
    ##Number of roomates, roomates probably have the same fait, however I don't sure if this is OK for persons with a lot of cabins
    roomates=train.groupby(['Cabin'])['Survived'].sum()
    roomates_dic=dict(zip(roomates.index,roomates.values))
    all_data['Roomates']=all_data.apply(lambda element: (roomates_dic.get(element.Cabin)-1 if (roomates_dic.get(element.Cabin)>0) else roomates_dic.get(element.Cabin) ) if (str(element.Cabin)!='nan') else 0,axis=1)
    #need remove nan from cabin from the test then
    all_data.loc[all_data.Roomates.isnull(),('Roomates')]=0
    ##people with the same surename, however there are common surename so is not 100% perfect and can affect the prediction
    all_data['Surname']=all_data['Name'].map(lambda element: element.split(',')[0].strip())
    surename_list=sorted(all_data['Surname'].unique(), key=lambda x: x)
    all_data['Surname']=labelencoder_X.fit_transform(all_data['Surname'])
    all_data.Cabin.fillna('U',inplace=True)
    all_data['Cabin']=all_data['Cabin'].map(lambda cabin: cabin[0])
    all_data['Cabin']=labelencoder_X.fit_transform(all_data['Cabin'])

    return all_data[:len(train)],all_data[len(train):]
    
    
    
    

In [23]:
train,test=data_prep(data_set,test_set)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
len(train)

891

In [25]:
len(test)

418

In [122]:
X_train=train.drop(['Ticket','Name','Title'],axis=1)

In [123]:
survived_train=data_set['Survived']

In [124]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier


In [125]:
kf=StratifiedKFold(n_splits=5)
parameters={'n_estimators':[10,50,100,150,200],'max_depth':[3,4,5,6,7,8,9,10]}
parameters_grad_boosting={'n_estimators':[10,50,100,150,200]}
parameters_xgb={'max_depth':[2,3],'learning_rate':[0.01,0.04,0.05],'n_estimators':[40,50]}

In [126]:

clf=RandomForestClassifier()
clfgb=GradientBoostingClassifier()
clfxgb=XGBClassifier()

In [127]:
from sklearn.model_selection import GridSearchCV

In [128]:
grid_search=GridSearchCV(clf,param_grid=parameters,cv=kf)
grid_searchgb=GridSearchCV(clfgb,param_grid=parameters_grad_boosting,cv=kf)
grid_searchxgb=GridSearchCV(clfgb,param_grid=parameters_xgb,cv=kf)

In [129]:
#X_train['Embarked']=labelencoder_X.fit_transform(X_train['Embarked'])
#X_train['Sex']=labelencoder_X.fit_transform(X_train['Sex'])


In [130]:
grid_search.fit(X_train,survived_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 50, 100, 150, 200], 'max_depth': [3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [131]:
#X_train.isnull().values.any()

In [132]:
#nan_rows = X_train[X_train.isnull().any(1)]

In [133]:
grid_search.best_score_

0.84511784511784516

In [134]:
grid_search.best_params_

{'max_depth': 9, 'n_estimators': 100}

In [135]:
clf_new=RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'],max_depth=grid_search.best_params_['max_depth'])

In [136]:
clf_new=clf_new.fit(X_train,survived_train)

In [137]:
y_test_predicted=clf_new.predict(X_train)

In [138]:
y_test=test.drop(['Ticket','Name','Title'],axis=1)

In [139]:
#y_test['Embarked']=labelencoder_X.fit_transform(y_test['Embarked'])

In [140]:
survived_prediction=clf_new.predict(y_test)

In [141]:
submission=pd.DataFrame({"PassengerID":y_test['PassengerId'],"Survived":survived_prediction})

In [142]:
submission.to_csv('random_forrest_submission_with_gridsearch_and_more_features_with_fare_and_roomates_and_cabin.csv',index=False)

In [143]:
train['Cabin'].unique()

array([8, 2, 4, 6, 3, 0, 1, 5, 7])

In [144]:
train['Cabin'].describe()

count    891.000000
mean       6.716049
std        2.460739
min        0.000000
25%        8.000000
50%        8.000000
75%        8.000000
max        8.000000
Name: Cabin, dtype: float64

In [145]:
train[train['Cabin']=='C23 C25 C27']

TypeError: invalid type comparison

In [ ]:
train['Cabin'].value_counts()

In [ ]:
roomates=data_set.groupby(['Cabin'])['Survived'].sum()

In [ ]:
data_set[data_set['Cabin']=='B18']

In [ ]:
roomates_dic=dict(zip(roomates.index,roomates.values))

In [ ]:
roomates_dic

In [ ]:
np.isnan(data_set.iloc[0].Cabin)
data_set['Roomates']=data_set.apply(lambda element: (roomates_dic.get(element.Cabin)-1 if (roomates_dic.get(element.Cabin)>0) else roomates_dic.get(element.Cabin) ) if (str(element.Cabin)!='nan') else 0,axis=1)

In [ ]:
data_set['Roomates'].describe()

In [ ]:
grid_searchgb.fit(X_train,survived_train)

In [ ]:
grid_searchgb.best_score_

In [ ]:
grid_searchgb.best_params_

In [ ]:
grid_searchxgb.fit(X_train,survived_train)

In [ ]:
grid_searchxgb.best_score_

In [ ]:
grid_searchxgb.best_params_

In [ ]:
rf=RandomForestClassifier(n_estimators=150, max_depth=6).fit(X_train,survived_train)

In [ ]:
gb=GradientBoostingClassifier(n_estimators=10).fit(X_train,survived_train)

In [ ]:
xgb=XGBClassifier(learning_rate=0.05,max_depth=2,n_estimators=50).fit(X_train,survived_train)

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
vote=VotingClassifier(estimators=[('rf',rf),('gb',gb),('xgb',xgb)],voting='soft',weights=[1,1,1])

In [ ]:
vote_final=vote.fit(X_train,survived_train)

In [ ]:
survived_predictionv=vote.predict(y_test)

In [ ]:
submission=pd.DataFrame({"PassengerID":y_test['PassengerId'],"Survived":survived_prediction})

In [ ]:
submission.to_csv('vote4.csv',index=False)

In [120]:
data_set['Ticket'].describe()

count          891
unique         681
top       CA. 2343
freq             7
Name: Ticket, dtype: object

In [121]:
data_set['Ticket'].value_counts()

CA. 2343             7
347082               7
1601                 7
347088               6
CA 2144              6
3101295              6
382652               5
S.O.C. 14879         5
PC 17757             4
4133                 4
LINE                 4
349909               4
347077               4
W./C. 6608           4
113760               4
2666                 4
113781               4
19950                4
17421                4
347742               3
239853               3
110152               3
24160                3
13502                3
PC 17572             3
C.A. 34651           3
248727               3
35273                3
F.C.C. 13529         3
PC 17755             3
                    ..
330877               1
315094               1
236852               1
4137                 1
4136                 1
226593               1
4134                 1
374746               1
110813               1
C.A. 6212            1
349219               1
34218                1
343120     